#### [198\. 打家劫舍](/problems/house-robber/)

你是一个专业的小偷，计划偷窃沿街的房屋。每间房内都藏有一定的现金，影响你偷窃的唯一制约因素就是相邻的房屋装有相互连通的防盗系统，**如果两间相邻的房屋在同一晚上被小偷闯入，系统会自动报警**。

给定一个代表每个房屋存放金额的非负整数数组，计算你 **不触动警报装置的情况下** ，一夜之内能够偷窃到的最高金额。

**示例 1：**

```
输入：[1,2,3,1]
输出：4
解释：偷窃 1 号房屋 (金额 = 1) ，然后偷窃 3 号房屋 (金额 = 3)。
     偷窃到的最高金额 = 1 + 3 = 4 。
```

**示例 2：**

```
输入：[2,7,9,3,1]
输出：12
解释：偷窃 1 号房屋 (金额 = 2), 偷窃 3 号房屋 (金额 = 9)，接着偷窃 5 号房屋 (金额 = 1)。
     偷窃到的最高金额 = 2 + 9 + 1 = 12 。
```

**提示：**

*   `1 <= nums.length <= 100`
*   `0 <= nums[i] <= 400`

In [ ]:
# 自定向下计算 记忆化搜索
class Solution:
    def rob(self, nums: list[int]) -> int:
        n = len(nums)
        # 记忆数组
        li = [-1] * n
        def dfs(i: int) -> int:
            if i < 0:
                return 0
            fro = li[i - 1] if i - 1 >= 0 and li[i - 1] != -1 else dfs(i - 1)
            pre = li[i - 2] if i - 2 >= 0 and li[i - 2] != -1 else dfs(i - 2)
            res =  max(fro, pre + nums[i])
            li[i] = res
            return res
        return dfs(n-1)

In [ ]:
# 自低向上 递推
class Solution:
    def rob(self, nums: list[int]) -> int:
        n = len(nums)
        if n > 1:
            nums[1] = max(nums[0], nums[1])
        for i in range(2, n):
            nums[i] = max(nums[i-1], nums[i-2] + nums[i])
        return nums[-1]

#### 0-1背包
有$n$个物品，第$i$个物品的体积为$w[i]$，价值为$v[i]$，每个物品至多选一个，求体积不超过$capacity$时的最大价值和
**回溯三问**
+ 当前操作：枚举第$i$个物品选或不选
  + 不选，剩余容量不变
  + 选，剩余容量减少$w[i]$
+ 子问题：在剩余容量为$c$时，从前$i$个物品中的到的最大和
+ 下一个子问题：分类讨论
  + 不选：在剩余容量为$c$时，从前$i - 1$个物品中得到的最大价值和
  + 选：在剩余容量为$c-w[i]$时，从前$i-1$个物品中得到的最大价值和
$dfs(i, c) = max(dfs(i-1, c), dfs(i-1, c-w[i]) + v[i])$

#### [416. 分割等和子集](https://leetcode.cn/problems/partition-equal-subset-sum/)
给你一个整数数组 `nums` 和一个整数 `target` 。

向数组中的每个整数前添加 `'+'` 或 `'-'` ，然后串联起所有整数，可以构造一个 **表达式** ：

*   例如，`nums = [2, 1]` ，可以在 `2` 之前添加 `'+'` ，在 `1` 之前添加 `'-'` ，然后串联起来得到表达式 `"+2-1"` 。

返回可以通过上述方法构造的、运算结果等于 `target` 的不同 **表达式** 的数目。

**示例 1：**

```
输入：nums = \[1,1,1,1,1\], target = 3
输出：5
解释：一共有 5 种方法让最终目标和为 3 。
-1 + 1 + 1 + 1 + 1 = 3
+1 - 1 + 1 + 1 + 1 = 3
+1 + 1 - 1 + 1 + 1 = 3
+1 + 1 + 1 - 1 + 1 = 3
+1 + 1 + 1 + 1 - 1 = 3

```

**示例 2：**

```
输入：nums = \[1\], target = 1
输出：1

```

**提示：**

*   `1 <= nums.length <= 20`
*   `0 <= nums[i] <= 1000`
*   `0 <= sum(nums[i]) <= 1000`
*   `-1000 <= target <= 1000`

假设所有加+的数字之和为$p$
则加符号的数字之和为$-sum(nums) + p$
则$target = p + (-s + p) = 2p - s$
$\Rightarrow p = (t + s) // 2$
问题转化为选任意元素使其和为$p = (t + s) // 2$

In [ ]:
class Solution:
    def findTargetSumWays(self, nums: list[int], target: int) -> int:
        target += sum(nums)
        if target < 0 or target % 2:
            return 0
        target >>= 1
        n = len(nums)
        # 元素为非负，因此初始化为-1
        cache = [[-1 for _ in range(target + 1)] for _ in range(n)]
        def dfs(i, c):
            if i < 0:
                # 如果刚好减完测可以构造返回1
                return 1 if c == 0 else 0
            a = cache[i-1][c] if i > 0 and c >= 0 and cache[i-1][c] != -1 else dfs(i-1, c)
            if c < nums[i]:
                # 容量比当前数小，当前不选
                cache[i][c] = a
                return a
            b = cache[i-1][c-nums[i]] if i > 0 and c-nums[i] > 0 and cache[i-1][c-nums[i]] != -1 else dfs(i-1, c-nums[i])
            # 否则返回当前选可以达成的结果数 加上不选的结果数
            cache[i][c] = a + b
            return a + b
        return dfs(n - 1, target)

##### c++写法
```cpp
class Solution {
public:
    bool dfs(int i, int c, vector<int> &nums, vector<vector<int>> &cache) {
        if (i < 0) return c == 0 ? 1 : 0;
        int a, b;
        a = (i > 0 && c >= 0 &&cache[i-1][c] != -1)? cache[i-1][c]: dfs(i-1, c, nums, cache);
        if (i > 0) cache[i-1][c] == a;
        if (c < nums[i]) return a;
        b = (i > 0 && c - nums[i] >= 0 && cache[i-1][c-nums[i]] != -1)? cache[i-1][c-nums[i]]: dfs(i-1, c-nums[i], nums, cache);
        if (i > 0) cache[i-1][c-nums[i]] = b;
        return a | b;
    }

    bool canPartition(vector<int>& nums) {
        int target = accumulate(nums.begin(), nums.end(), 0);
        int n = nums.size();
        if (target & 1) return false;
        target >>= 1;
        vector<vector<int>> cache(n+1, vector<int>(target + 1, -1));
        return dfs(nums.size() - 1, target, nums, cache);
    }
};
```

In [ ]:
# 递推
class Solution:
    def findTargetSumWays(self, nums: list[int], target: int) -> int:
        target += sum(nums)
        if target < 0 or target % 2:
            return 0
        target >>= 1
        n = len(nums)
        # 存结果 默认为0
        li = [[0 for _ in range(target + 1)] for _ in range(n + 1)]
        li[0][0] = 1
        for i, x in enumerate(nums):
            for c in range(target + 1):
                if c < x:
                    li[i+1][c] = li[i][c]
                else:
                    li[i+1][c] = li[i][c] + li[i][c-x]
        return li[n][target]


In [ ]:
# 优化空间
class Solution:
    def findTargetSumWays(self, nums: list[int], target: int) -> int:
        target += sum(nums)
        if target < 0 or target % 2:
            return 0
        target >>= 1
        n = len(nums)
        # 存结果 默认为0
        li = [0 for _ in range(target + 1)]
        li[0] = 1
        for i in nums:
            # 0-1背包需要逆推
            for c in range(target, i-1, -1):
                li[c] = li[c] + li[c-i]
        return li[target]

#### 完全背包
有$n$个物品，第$i$个物品的体积为$w[i]$，价值为$v[i]$，每个物品**无限次重复选**，求体积不超过$capacity$时的最大价值和
**回溯三问**
+ 当前操作：枚举第$i$种物品选或不选
  + 不选，剩余容量不变
  + 选，剩余容量减少$w[i]$
+ 子问题：在剩余容量为$c$时，从前$i$种物品中的到的最大和
+ 下一个子问题：分类讨论
  + 不选：在剩余容量为$c$时，从前$i-1$种物品中得到的最大价值和
  + 选：在剩余容量为$c-w[i]$时，从前$i$种物品中得到的最大价值和
$dfs(i, c) = max(dfs(i-1, c), dfs(i-1, c-w[i]) + v[i])$

#### [322. 零钱兑换](https://leetcode.cn/problems/coin-change)

In [3]:
# 递归
from functools import cache
class Solution:
    def coinChange(self, coins: list[int], amount: int) -> int:
        n = len(coins)
        @cache
        def dfs(i, c):
            if i < 0:
                return 0 if c == 0 else float("inf")
            if c < coins[i]:
                return dfs(i-1, c)
            return min(dfs(i-1, c), dfs(i, c-coins[i])+ 1)
        a = dfs(n-1, amount)
        return a if a < float("inf") else -1

In [ ]:
# 递推
class Solution:
    def coinChange(self, coins: list[int], amount: int) -> int:
        n = len(coins)
        dp = [[float("inf") for _ in range(amount+1)] for _ in range(n+1)]
        dp[0][0] = 0
        for i, val in enumerate(coins):
            for c in range(amount+1):
                if c < val:
                    dp[i+1][c] = dp[i][c]
                else:
                    dp[i+1][c] = min(dp[i][c], dp[i+1][c-val] + 1)
        a = dp[n][amount]
        return a if a < float("inf") else -1

In [4]:
from functools import cache

# 请在此输入您的代码
li = [6, 2, 5, 5, 4, 5, 6, 3, 7, 6]
num = [i for i in range(10)]

def dfs(i, c, n):
    if i < 0:
        return n if c >= 0 else 0
    a = dfs(i-1, c, n)
    n *= 10
    n += num[i]
    if c < li[i]:
        return a
    b = dfs(i, c-li[i], n)
    return max(a, b)

print(dfs(9, 300, 0))

KeyboardInterrupt: 